In [1]:
## imports
import sys
import os
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import eofs
from eofs.xarray import Eof
import cftime
import nc_time_axis

import mtspec

# from pylab import *

import datetime
import time
import glob, os
import math
import matplotlib.colors as colors
import xesmf as xe
from defs import regularAtlmask

In [1]:
datafolder = ''

In [14]:
experiment = 'Eoi400' 
model = 'NorESM1-F'
folder = datafolder+'Raw_data/'+model+'/'

In [15]:
if experiment == 'Eoi400':
    dsV = xr.open_dataset(folder+experiment+'/PlioMIP2_03.micom.hy.2400-2499mean.vvellvl.nc')
    dsT = xr.open_dataset(folder+experiment+'/PlioMIP2_03.micom.hy.2400-2499mean.templvl.nc')
    dsS = xr.open_dataset(folder+experiment+'/PlioMIP2_03.micom.hy.2400-2499mean.salnlvl.nc')
    dsr = xr.open_dataset(folder+experiment+'/grid.nc')
else:
    dsV = xr.open_dataset(folder+experiment+'/NBF1850OC_f19_tn11_02.micom.hy.1900-1999mean.vvellvl.nc')
    dsT = xr.open_dataset(folder+experiment+'/NBF1850OC_f19_tn11_02.micom.hy.1900-1999mean.templvl.nc')
    dsS = xr.open_dataset(folder+experiment+'/NBF1850OC_f19_tn11_02.micom.hy.1900-1999mean.salnlvl.nc')
    dsr = xr.open_dataset(folder+experiment+'/grid.nc')

dsV['lat']=dsr.vlat
dsV['lon']=dsr.vlon

dsT['lat']=dsr.plat
dsT['lon']=dsr.plon

dsS['lat']=dsr.plat
dsS['lon']=dsr.plon

dsgrid = xr.open_dataset(folder+experiment+'/'+experiment+'.SST.mean_month.nc', decode_times=False)

In [16]:
regridder = xe.Regridder(dsS, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsS.salnlvl[0])

ds = xr.Dataset(data_vars = dict(so=(("z","lat","lon"), var)), coords = dict(z = var.depth.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/so_1x1_100yr.nc')

In [17]:
regridder = xe.Regridder(dsV, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsV.vvellvl[0])

ds = xr.Dataset(data_vars = dict(vo=(("z","lat","lon"), var)), coords = dict(z = var.depth.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/vo_1x1_100yr.nc')

In [18]:
regridder = xe.Regridder(dsT, dsgrid, 'bilinear', periodic = True, ignore_degenerate=True)

var = regridder(dsT.templvl[0])

ds = xr.Dataset(data_vars = dict(thetao=(("z","lat","lon"), var)), coords = dict(z = var.depth.values, lat = var.latitude.values, lon = var.longitude.values))
xr.Dataset.to_netcdf(ds, path = datafolder+'Processed/'+model+'/'+experiment+'/thetao_1x1_100yr.nc')